# EasyEdit Example with T5 and KN

The following example shows how to use KN edit with T5.

## Preparation

Except for installing all the necessary dependencies, you need to download the pre-trained language model. The default hparams in the 'hparams' folder use huggingface cache in default. You can change the 'model_name' in hparam file to the model name to let huggingface automatically download the model for you. In our example, it means changing the 'model_name' to t5-3B.

```python
alg_name: "KN"
model_name: "./hugging_cache/t5-3B"
device: 0
model_parallel: false

lr_scale: 1.0
n_toks: 10
refine: false
batch_size: 1
steps: 20
adaptive_threshold: 0.2
p: 0.4
```

In [1]:
%cd ..

/mnt/8t/xkw/EasyEdit


In [2]:
from easyeditor import KNHyperParams
from easyeditor import BaseEditor
import os

hparams = KNHyperParams.from_hparams("./hparams/KN/t5-3B.yaml")

prompts = ['Who was the designer of Lahti Town Hall?']
target_new = ['Joe Biden']
subject = ['Lahti Town Hall']

editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    target_new=target_new,
    subject=subject,
    sequential_edit=True
)
print(metrics)
print(type(edited_model))

2024-12-01 14:36:21,274 - easyeditor.editors.editor - INFO - Instantiating model
12/01/2024 14:36:21 - INFO - easyeditor.editors.editor -   Instantiating model
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/mnt/8t/xkw/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transfo


61 coarse neurons found - refining
61 neurons remaining after refining

Before modification - groundtruth probability: tensor([2.5646e-29], device='cuda:0')
Argmax completion: `</s></s><extra_id_5>`
Argmax prob: 0.21098077093542403


100%|██████████| 1/1 [08:03<00:00, 483.76s/it]


After modification - groundtruth probability: tensor([4.9062e-22], device='cuda:0')
Argmax completion: `dendendenden`
Argmax prob: 0.9392140688144989



2024-12-01 14:47:10,575 - easyeditor.editors.editor - INFO - 0 editing: Who was the designer of Lahti Town Hall? -> Joe Biden  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}
12/01/2024 14:47:10 - INFO - easyeditor.editors.editor -   0 editing: Who was the designer of Lahti Town Hall? -> Joe Biden  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}


Metrics Summary:  {'pre': {'rewrite_acc': 0.0}, 'post': {'rewrite_acc': 0.3333333432674408}}
[{'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Who was the designer of Lahti Town Hall?', 'target_new': 'Joe Biden', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'subject': 'Lahti Town Hall'}, 'post': {'rewrite_acc': [0.3333333432674408], 'locality': {}, 'portability': {}}}]
<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


## Compare edited model and original model

In [4]:
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer

device = 1
tokenizer = T5Tokenizer.from_pretrained("./hugging_cache/t5-3B")
origin_model = T5ForConditionalGeneration.from_pretrained("./hugging_cache/t5-3B").to(f"cuda:{device}")


In [8]:

batch = tokenizer(prompts, return_tensors='pt', padding=True)

origin_output = origin_model.generate(
    input_ids=batch['input_ids'].to(origin_model.device),
    attention_mask=batch['attention_mask'].to(origin_model.device),
    max_new_tokens=20
)

output = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    max_new_tokens=5
)


for i in range(len(prompts)):
    print(f'Prompt: {prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode(origin_output[i], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(output[i], skip_special_tokens=True)}')
    print('--'*50 )


Prompt: Who was the designer of Lahti Town Hall?
Pre-Edit  Output: was the designer of Lahti Town Hall??? Who was the designer
Post-Edit Output: dendendenden
----------------------------------------------------------------------------------------------------
